In [ ]:
using CSV, DataFrames


In [2]:
using CSV, DataFrames
using CairoMakie
using LinearAlgebra

# Read the CSV file
df = CSV.File("Z_GR_NPHI_RHOB_CALI.txt",delim=',', ignorerepeated=true) |> DataFrame

# Convert columns to arrays
z = df[:, 1] .* 0.0003048  # Convert to kilometers
γ = df[:, 2]
ϕ = df[:, 3]
δ = df[:, 4]
cali = df[:, 5]
n = length(z);

# Define ranges for CALI, velocity, density, and porosity
# For plotting and the later PDF
cali_range = (5.9, 6.3) # CALI     min=5.9, max=6.3
v_x_range  = (3.0, 7.0) # Velocity min=3.0, max=7.0
δ_x_range  = (2.3, 2.8) # Density  min=2.3, max=2.8
ϕ_x_range  = (0.0, 0.3) # Porosity min=0.0, max=0.3
γ_x_range  = (0  , 200);# Gamma    min=0.0, max=200

In [ ]:
fig = Figure(size=(800, 600))

ax1 = Axis(fig[1, 1], xlabel=L"\gamma\text{(API)}", ylabel=L"z\text{(km)}", yreversed=true)
ax2 = Axis(fig[1, 2], xlabel=L"CALI\text{(in)}", yreversed=true)
ax3 = Axis(fig[1, 3], xlabel=L"\phi\text{(.)}", yreversed=true)
ax4 = Axis(fig[1, 4], xlabel=L"\delta\text{(g/cm)}^3", yreversed=true)
lines!(ax1, γ, z, color=:black)
xlims!(ax1, γ_x_range)
lines!(ax2, cali, z, color=:green)
xlims!(ax2, cali_range)
lines!(ax3, ϕ, z, color=:red)
xlims!(ax3, ϕ_x_range)
lines!(ax4, δ, z, color=:blue)
xlims!(ax4, δ_x_range)
linkyaxes!(ax1, ax2, ax3, ax4)
ylims!(ax4, maximum(z), minimum(z), )

fig

In [ ]:
# Compute dispersion for delta and phi
cmax = 8  # Max allowed caliper
cref = 6  # Reference caliper
a_δ = 0.05  # Minimum uncertainty
a_ϕ = 0.03 # Minimum uncertainty
b_δ = range(0.04, stop=0.05, length=length(cali))
b_ϕ = range(0.05, stop=0.06, length=length(cali))

# Compute σ_δ, σ_ϕ and define σ_v
σ_δ = a_δ .+ (cali .- cref) ./ (cmax - cref) .* b_δ
σ_ϕ = a_ϕ .+ (cali .- cref) ./ (cmax - cref) .* b_ϕ
σ_v = 0.5

In [96]:
function gaussian_1d(σ, x, μ, n)
    # Replicate x to match n rows
    x_tiled = repeat(x', n, 1)  # Transpose x and repeat n times
    # Compute Gaussian function
    g = (1 ./ (σ .* sqrt(2 * π))) .* exp.(-0.5 * ((x_tiled .- μ) .^ 2) ./ (σ .^ 2))
    return g
end

# Define parameters
n_x = 100  # Number of values for dispersion

# Generate linearly spaced vectors
v_x = range(v_x_range..., length=n_x) |> collect
δ_x = range(δ_x_range..., length=n_x) |> collect
ϕ_x = range(ϕ_x_range..., length=n_x) |> collect;
v = 5.654 .- 0.008 .* γ # Compute v using the GR-v equation

# Define marginal PDFs using the gaussian_1d function
δ_marginal_prior = gaussian_1d(σ_δ, δ_x, δ, n)
v_marginal_prior = gaussian_1d(σ_v, v_x, v, n)
ϕ_marginal_prior = gaussian_1d(σ_ϕ, ϕ_x, ϕ, n);

In [ ]:
fig = Figure(size=(600, 600))
ax1 = Axis(fig[1, 1], xlabel=L"v\text{(km/s)}", ylabel=L"z\text{(km)}", yreversed=true)
ax2 = Axis(fig[1, 2], xlabel=L"\phi\text{(.)}", yreversed=true)
ax3 = Axis(fig[1, 3], xlabel=L"\delta\text{(g/cm)}^3", yreversed=true)
heatmap!(ax1, v_x, z, v_marginal_prior')
lines!(ax1, v, z, color=:black, linewidth=0.75)
xlims!(ax1, v_x_range)
heatmap!(ax2, ϕ_x, z, ϕ_marginal_prior')
lines!(ax2, ϕ, z, color=:black, linewidth=0.75)
xlims!(ax2, ϕ_x_range)
heatmap!(ax3, δ_x, z, δ_marginal_prior')
lines!(ax3, δ, z, color=:black, linewidth=0.75)
xlims!(ax3, δ_x_range)
linkyaxes!(ax1, ax2, ax3)
ylims!(ax3, maximum(z), minimum(z))
fig

In [ ]:
# Extract marginal PDFs for a specific index
index = 1600
v_marginal_prior_16 = v_marginal_prior[index, :]
ϕ_marginal_prior_16 = ϕ_marginal_prior[index, :]
δ_marginal_prior_16 = δ_marginal_prior[index, :]



# Plot marginal PDFs
fig, axes = plt.subplots(3, 1, figsize=(8, 8))
axes[0].plot(v_x, g_v_marginal)
axes[0].set_title(f"Marginal PDF at depth {z[1600]:.2f} km")
axes[0].set_xlabel('Velocity (km/s)')
axes[1].plot(delta_x, g_delta_marginal)
axes[1].set_xlabel('Density (g/cm^3)')
axes[2].plot(phi_x, g_phi_marginal)
axes[2].set_xlabel('Porosity')
plt.tight_layout()
plt.show()

# Create a 3D grid for joint PDF calculation
v_grid, delta_grid, phi_grid = np.meshgrid(g_v_marginal, g_delta_marginal, g_phi_marginal, indexing='ij')

# Calculate the joint PDF as the product of marginals
g_joint_prior = v_grid * delta_grid * phi_grid

In [ ]:
lines(δ_marginal_prior[1,:])